Load netCDF files in Tensorflow

In [1]:
import numpy
import netCDF4 as nc
import xarray
import tensorflow_datasets as tfds
import tensorflow as tf
import os
import shutil
import glob
import plotly.express as px


print("Tensorflow:", tf.version.VERSION)
print("Xarray:", xarray.__version__)
print("netCDF4:", nc.__version__)

/media/jtrvz/chugchug/Git/drought-detector/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-17 11:08:15.215549: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-17 11:08:15.284230: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-17 11:08:15.535607: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-17 11:08:16.480567: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find Ten

Tensorflow: 2.16.1
Xarray: 2024.3.0
netCDF4: 1.6.5


In [2]:
temp_files_dir = "/media/jtrvz/1tb/drought_data/temperature/era5/Germany/monthly/netcdf/avg"
prec_files_dir = "/media/jtrvz/1tb/drought_data/precipitation/era5/Germany/monthly/netcdf/avg"

Generator method

In [3]:
def load_nc_dir_with_generator(dir_):
    def gen():
        for file in glob.glob(os.path.join(dir_, "*.nc")):
            ds = xarray.open_dataset(file, engine='netcdf4')
            yield {key: tf.convert_to_tensor(val) for key, val in ds.items()}


    sample = next(iter(gen()))

    return tf.data.Dataset.from_generator(
        gen,
        output_signature={
            key: tf.TensorSpec(val.shape, dtype=val.dtype)
            for key, val in sample.items()
        }
    )

Load to tfrecords

In [4]:
def load_nc_dir_cached_to_tfrecord(dir_, var, save_location):
    """
    Save data to tfRecord, open it, and deserialize.
    
    Parameters
    :param dir_: directory with netCDF files
    :param var: variable to extract from netCDF files

    """
    generator_tfds = load_nc_dir_with_generator(dir_)
    writer = tf.data.experimental.TFRecordWriter(save_location)
    writer.write(generator_tfds.map(lambda x: tf.io.serialize_tensor(x[var])))

    return tf.data.TFRecordDataset(save_location).map(
        lambda x: tf.io.parse_tensor(x, tf.float64))

Example

In [5]:
temp_tfrecord = load_nc_dir_cached_to_tfrecord(temp_files_dir, "t2m", "temp.tfrecord")
prec_tfrecord = load_nc_dir_cached_to_tfrecord(prec_files_dir, "tp", "prec.tfrecord")

Instructions for updating:
To write TFRecords to disk, use `tf.io.TFRecordWriter`. To save and load the contents of a dataset, use `tf.data.experimental.save` and `tf.data.experimental.load`
